In [6]:
from enbios.generic.mermaid2hierarchy import convert_mermaid_lines
import json

In [2]:
mermaid1="""graph BT
b1-->a
b2-->a"""

In [4]:
hierarchy = convert_mermaid_lines(mermaid1)
hierarchy

{'name': 'a',
 'aggregator': None,
 'config': None,
 'children': [{'name': 'b1', 'adapter': None, 'config': None},
  {'name': 'b2', 'adapter': None, 'config': None}]}

In [7]:
print(json.dumps(hierarchy, indent=2, ensure_ascii=False))

{
  "name": "a",
  "aggregator": null,
  "config": null,
  "children": [
    {
      "name": "b1",
      "adapter": null,
      "config": null
    },
    {
      "name": "b2",
      "adapter": null,
      "config": null
    }
  ]
}


In [9]:
# Specifying adapters and aggregators wtih ':::' (which can be done once per node)

In [13]:
mermaid2="""graph BT
b1-->a:::sum
b2:::bw-->a"""

hierarchy = convert_mermaid_lines(mermaid2)
hierarchy

{'name': 'a',
 'aggregator': 'sum',
 'config': None,
 'children': [{'name': 'b1', 'adapter': None, 'config': None},
  {'name': 'b2', 'adapter': 'bw', 'config': None}]}

In [14]:
mermaid2b = """
graph BT
    b1 --> a:::sum
    b2 -- bw --> a
"""

print(json.dumps(convert_mermaid_lines(mermaid2), indent=2, ensure_ascii=False))

{
  "name": "a",
  "aggregator": "sum",
  "config": null,
  "children": [
    {
      "name": "b1",
      "adapter": null,
      "config": null
    },
    {
      "name": "b2",
      "adapter": "bw",
      "config": null
    }
  ]
}


In [15]:
mermaid2="""graph BT
a:::sum
b1:::bw
b2:::bw
b1-->a
b2-->a"""

print(json.dumps(convert_mermaid_lines(mermaid2), indent=2, ensure_ascii=False))

{
  "name": "a",
  "aggregator": "sum",
  "config": null,
  "children": [
    {
      "name": "b1",
      "adapter": "bw",
      "config": null
    },
    {
      "name": "b2",
      "adapter": "bw",
      "config": null
    }
  ]
}


In [19]:
mermaid3="""graph BT
a:::sum
b1:::bw,code:xxx1
b2:::bw,code:xxx2

b1-->a
b2-->a"""

print(json.dumps(convert_mermaid_lines(mermaid3), indent=2, ensure_ascii=False))

{
  "name": "a",
  "aggregator": "sum",
  "config": null,
  "children": [
    {
      "name": "b1",
      "adapter": "bw",
      "config": {
        "code": "xxx1"
      }
    },
    {
      "name": "b2",
      "adapter": "bw",
      "config": {
        "code": "xxx2"
      }
    }
  ]
}
